# 04: Advanced Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/04_advanced_analysis.ipynb)

This notebook covers advanced snowpack analysis techniques including gradient analysis, temporal patterns, and comparing multiple locations.

## What You'll Learn

- Density gradient analysis to identify weak layers
- Temperature gradient analysis for understanding metamorphism
- Temporal analysis with rolling windows and rates of change
- Comparing multiple locations and scenarios


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:

%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow



In [ ]:
import xsnow
import numpy as np
import matplotlib.pyplot as plt

# Load sample data
print("Loading xsnow sample data...")
try:
    ds = xsnow.single_profile_timeseries()
    print("✅ Data loaded successfully")
    print(f"Dataset dimensions: {dict(ds.dims)}")
except Exception as e:
    print(f"⚠️ Error loading sample data: {e}")
    print("Make sure xsnow is properly installed:")
    print("  pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
    raise

%matplotlib inline


## Part 1: Density Gradient Analysis

Density gradients can help identify weak layers in the snowpack. Large changes in density with depth may indicate potential weak layers.


**Now You Try**: After running the gradient analysis above, try:
- Computing density gradient for a different time step
- Finding layers with density gradient > 50 kg/m³ per layer
- Comparing density gradients between different locations


In [ ]:
# Compute density gradient (change in density with depth)
# Large gradients can indicate weak layers
density_gradient = ds['density'].diff(dim='layer')
print(f"Density gradient shape: {density_gradient.shape}")

# Select a single profile to visualize
profile = ds.isel(location=0, time=-1, slope=0, realization=0)
depth = -profile.coords['z'].values
density = profile['density'].values
gradient = density_gradient.isel(location=0, time=-1, slope=0, realization=0).values

# Plot density and gradient
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.plot(density, depth, 'b-', linewidth=2)
ax1.set_xlabel('Density (kg/m³)')
ax1.set_ylabel('Depth (m)')
ax1.set_title('Density Profile')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

ax2.plot(gradient, depth[:-1], 'r-', linewidth=2)
ax2.axvline(x=0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Density Gradient (kg/m³ per layer)')
ax2.set_ylabel('Depth (m)')
ax2.set_title('Density Gradient')
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()



### Computing Temperature Gradient

First, let's compute the temperature gradient:


In [ ]:
# Temperature gradient = change in temperature per unit depth
# Large gradients (> 10 K/m) can lead to faceting (weak layer formation)

# Compute gradient using z coordinate
temp_diff = ds['temperature'].diff(dim='layer')
z_diff = ds.coords['z'].diff(dim='layer')
temp_gradient = temp_diff / z_diff

# Find layers with high gradients (> 10 K/m indicates faceting)
high_gradient = temp_gradient > 10  # K/m
n_high_gradient = high_gradient.sum().values
print(f"Found {n_high_gradient} layers with temperature gradient > 10 K/m")

# Visualize temperature gradient for a single profile
profile = ds.isel(location=0, time=-1, slope=0, realization=0)
depth = -profile.coords['z'].values
tg_profile = temp_gradient.isel(location=0, time=-1, slope=0, realization=0)
temp_profile = ds['temperature'].isel(location=0, time=-1, slope=0, realization=0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.plot(temp_profile.values, depth, 'b-', linewidth=2)
ax1.set_xlabel('Temperature (°C)')
ax1.set_ylabel('Depth (m)')
ax1.set_title('Temperature Profile')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

ax2.plot(tg_profile.values, depth[:-1], 'r-', linewidth=2)
ax2.axvline(x=10, color='k', linestyle='--', alpha=0.5, label='10 K/m threshold')
ax2.set_xlabel('Temperature Gradient (K/m)')
ax2.set_ylabel('Depth (m)')
ax2.set_title('Temperature Gradient')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()


## Part 2: Temperature Gradient Analysis

Temperature gradients are important for understanding metamorphism and weak layer formation:


### 📊 Proof by Example, Disproof by Counterexample

**Proof by Example, Disproof by Counterexample Pattern**: Let's explore common misconceptions about temperature gradients and weak layers.

**Example (Proof)**: High temperature gradients (> 10 K/m) often indicate faceting and weak layer formation. Let's see this in action:


In [ ]:
# Example: High temperature gradient leading to weak layer
# Find profiles with high temperature gradients
high_tg_profiles = temp_gradient > 10  # K/m

# Count how many layers have high gradients
n_high_tg = high_tg_profiles.sum().values
print(f"Found {n_high_tg} layers with temperature gradient > 10 K/m")
print("This supports the idea that high gradients can indicate weak layer formation.")

# Show an example profile with high gradient
if n_high_tg > 0:
    # Find first occurrence
    high_tg_locations = high_tg_profiles.any(dim=['time', 'layer'])
    print(f"\nExample: {high_tg_locations.sum().values} locations have high temperature gradients")


**Counterexample (Disproof)**: However, high temperature gradients don't ALWAYS mean weak layers. Sometimes other factors matter more. Let's explore:


In [ ]:
# Counterexample: High gradient but strong layer (due to other factors)
# Example: High gradient with high density might still be strong
high_tg_high_density = (temp_gradient > 10) & (ds['density'] > 300)
n_strong_high_tg = high_tg_high_density.sum().values
print(f"Found {n_strong_high_tg} layers with high temp gradient BUT high density (> 300 kg/m³)")
print("This shows that temperature gradient alone doesn't always indicate weak layers.")
print("Other factors (density, grain type, bonding) also matter!")


**Now You Try**: After running the temperature gradient analysis, try:
- Finding profiles where high temperature gradient (> 10 K/m) occurs with low density (< 150 kg/m³) - these are more likely weak layers
- Comparing temperature gradients between different time periods
- Plotting temperature gradient vs density to see the relationship


**Now You Try**: After comparing locations, try:
- Comparing a different metric (e.g., temperature gradient or SWE) across locations
- Finding which location has the most variable density over time
- Creating a visualization comparing multiple locations side-by-side


## Part 3: Comparing Multiple Locations

When you have data from multiple locations, you can compare their characteristics.


In [ ]:
# Compare mean density across locations
mean_density_by_location = ds['density'].mean(dim=['time', 'layer'])
print("Mean density by location:")
for i, loc in enumerate(ds.coords['location'].values):
    mean_dens = mean_density_by_location.isel(location=i).values
    print(f"  Location {loc}: {mean_dens:.1f} kg/m³")

# Compare snow height evolution
print("\nSnow height by location (last time step):")
for i, loc in enumerate(ds.coords['location'].values):
    hs_loc = ds['HS'].isel(location=i, slope=0, realization=0, time=-1)
    print(f"  Location {loc}: {hs_loc.values:.2f} m")


**Now You Try**: After running the temporal analysis, try:
- Computing a different rolling window (try 3-day, 14-day, or 30-day averages)
- Calculating the rate of change for a different variable (e.g., density or temperature)
- Finding the time period with the fastest snow height increase


## Part 4: Advanced Temporal Analysis

Time series analysis helps understand how the snowpack evolves over time.


In [ ]:
# Extract snow height time series
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
times = ds.coords['time'].values

# 7-day rolling average (smoothing)
hs_7day = hs_series.rolling(time=7, center=True).mean()

# Rate of change
hs_rate = hs_series.diff(dim='time')

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(times, hs_series.values, 'b-', alpha=0.5, label='Original')
ax1.plot(times, hs_7day.values, 'r-', linewidth=2, label='7-day rolling average')
ax1.set_xlabel('Time')
ax1.set_ylabel('Snow Height (m)')
ax1.set_title('Snow Height Time Series')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(times[1:], hs_rate.values[1:], 'g-', linewidth=2)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Time')
ax2.set_ylabel('Rate of Change (m/time step)')
ax2.set_title('Snow Height Rate of Change')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean rate of change: {hs_rate.mean().values:.4f} m/time step")


## Summary

In this notebook, you learned:

1. **Density gradients**: Identifying weak layers through density changes with depth
2. **Temperature gradients**: Understanding faceting conditions (gradients > 10 K/m indicate weak layer formation)
3. **Multi-location comparison**: Comparing statistics across different locations
4. **Temporal analysis**: Using rolling averages to smooth time series and calculating rates of change

## Key Techniques

- **Gradients**: `ds['variable'].diff(dim='layer')` for layer-to-layer differences
- **Temperature gradient**: `temp_diff / z_diff` to get gradient per unit depth
- **Rolling windows**: `ds.rolling(time=7).mean()` for smoothing time series
- **Rate of change**: `ds.diff(dim='time')` to see how values change over time
- **Multi-location stats**: `.mean(dim=['time', 'layer'])` to aggregate across dimensions

## Next Steps

- **05_working_with_custom_data.ipynb**: Load and analyze your own files
- **06_extending_xsnow.ipynb**: Create custom analysis functions

## Exercises

1. Compute temperature gradient and identify layers with gradients > 10 K/m
2. Calculate a 7-day rolling average of snow height for a different location
3. Compare mean density profiles across all locations in your dataset
4. Plot the rate of change of temperature at a specific depth over time
